# Item Based Collaborative Filtering

Suggests items by identifying relationships between different items based on user ratings.

This approach calculates the similarity between items using user ratings. Here, we assume that if users liked a particular item, they are more likely to enjoy items that are similar to it.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from coursemate.dataset import Dataset
from coursemate.model import ItemBasedCF

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)


In [2]:
dataset = Dataset('data/Coursera_courses.csv', 'data/Coursera.csv', 'data/Coursera_reviews.csv')
dataset.set_interaction_counts(3, 50)
dataset.show_dataset_details()

Loading Coursera courses...
Loading Coursera reviews...
Segmenting out students with less than 3 or more than 50 reviews...
30719 students, 468 courses, 174219 reviews
Sparsity: 1.21%
Duplicates: 4.54%


In [3]:
dataset.set_train_test_split_by_user()


Setting the train-test split by user...


In [3]:

# Instantiate the ItemBasedCF model
item_cf_model = ItemBasedCF(dataset.course_set)

# Fit the model with the training data
item_cf_model.fit(dataset.train_ratings)

Setting the train-test split by user...


In [6]:
# Use the model to get recommendations for a given user
user_previous_courses = tuple(dataset.train_ratings[dataset.train_ratings['reviewers'] == 'By Kelvin k']['course_id'])
recommendations = item_cf_model.recommend(user_previous_courses, k=5)

recommendations

['cybersecurity-roles-processes-operating-system-security',
 'python',
 'python-functions-files-dictionaries',
 'python-data',
 'technical-support-fundamentals']

In [5]:
train_Xmatrix, train_ymatrix, df_train_X, df_train_y = dataset.get_train_matrix_split(ratio=0.8)
test_Xmatrix, test_ymatrix, df_test_X, df_test_y = dataset.get_test_matrix_split(ratio=0.5)

Computing the training and test rating matrix...


131100it [00:07, 16610.99it/s]


Computing the test rating matrix split...
Computing the training and test rating matrix...


131100it [00:07, 16852.73it/s]
43119it [00:02, 17502.71it/s]


In [10]:
def calculate_hit_rate(model, test_X, test_y, k=5):
    hit_count = 0
    total = 0

    for user_id in tqdm(dataset.test_students):
        user_history = test_X[test_X['reviewers'] == user_id]['course_id'].values
        actual_next_courses = test_y[test_y['reviewers'] == user_id]['course_id'].values
        
        if len(actual_next_courses) == 0:
            continue

        recommended_courses = model.recommend(user_history, k=k)

        hits = any(course in recommended_courses for course in actual_next_courses)
        hit_count += 1 if hits else 0
        total += 1

    # Calculate overall hit rate
    return hit_count / total


item_based_cf_model = ItemBasedCF(dataset.course_set)

all_train = pd.concat([df_train_X, df_train_y])

# fit with both training X and training y
item_based_cf_model.fit(all_train)


In [11]:
calculate_hit_rate(item_based_cf_model, df_test_X, df_test_y, k=20)

100%|██████████| 7680/7680 [04:18<00:00, 29.72it/s]


0.6087239583333334

In [17]:
calculate_hit_rate(item_based_cf_model, df_test_X, df_test_y, k=5)

100%|██████████| 7680/7680 [04:02<00:00, 31.72it/s]


0.378515625

### Evaluation

In [3]:


def hit_rate(model, test_set, k=5):
    hits = 0
    total = 0
    #test_set['reviewers'].unique():
    for user_id in tqdm(test_set['reviewers'].unique()):
        # Get previously rated courses for the user in training data
        prev_courses = tuple(dataset.train_ratings[dataset.train_ratings['reviewers'] == user_id]['course_id'])

        # get the top-k recommended courses from the model
        recommended_courses = model.recommend(prev_courses, k)

        # Get the actual top-k rated courses for the user in the test data
        actual_top_k = test_set[test_set['reviewers'] == user_id].nlargest(k, 'rating')['course_id']

        # Calculate hits
        if any(course in recommended_courses for course in actual_top_k):
            hits += 1
        total += 1
        
        # print(f"Hit Rate: {hits / total if total > 0 else 0}")
        # if total == 500: # Stop after 500 users
        #     break

    return hits / total if total > 0 else 0


In [7]:
dataset.set_train_test_split_random(0.999)

Setting the train-test split randomly...


In [8]:
cf_model = ItemBasedCF(dataset.course_set)

cf_model.fit(dataset.train_ratings)

In [9]:
hit_rate(cf_model, dataset.test_ratings, k=10)

100%|██████████| 174/174 [00:06<00:00, 28.34it/s]


0.3390804597701149